In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nest_asyncio
import dateutil.parser
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
from cryptography.hazmat.primitives import serialization

# Import the provided client library
from clients.clients import KalshiHttpClient, Environment

# Plotting style
sns.set_theme(style="darkgrid")
%matplotlib inline

# --- CONFIGURATION ---
# Load your keys (Assumes you have a kalshi_key.txt or similar, or paste them here)
# NEVER share this notebook with your private keys hardcoded if you publish it.

# Initialize Client
try:
	nest_asyncio.apply()


	load_dotenv()
	env = Environment.PROD
	KEYID = os.getenv('KEYID')
	KEYFILE = os.getenv('KEYFILE')
	try:
		with open(KEYFILE, "rb") as key_file:
			private_key = serialization.load_pem_private_key(
				key_file.read(),
				password=None
			)
	except FileNotFoundError:
		raise FileNotFoundError(f"Private key file not found at {KEYFILE}")
	except Exception as e:
		raise Exception(f"Error loading private key: {str(e)}")

	client = KalshiHttpClient(
		key_id=KEYID,
		private_key=private_key,
		environment=env
	)
	print("✅ Client initialized successfully.")
except Exception as e:
	print(f"❌ Error initializing client: {e}")

✅ Client initialized successfully.


In [37]:
import pandas as pd
import numpy as np
import dateutil.parser

def fetch_market_candles_to_csv(market_tickers, filename, limit=1000):
	"""Fetches candlestick data and returns it as a pandas DataFrame."""

	all_candle_data = []

	for i, ticker in enumerate(market_tickers):
		print(f"({i+1}/{len(market_tickers)}) Fetching market data for Series: {ticker}...")

		# 1. Fetch Markets
		try:
			response = client.get_markets(
				series_ticker=ticker,
				status="settled",
				limit=limit
			)
		except Exception as e:
			print(f"❌ Failed to fetch markets for {ticker}: {e}")
			continue

		if 'markets' not in response:
			print(f"No markets found for {ticker}.")
			continue

		markets = response['markets']
		print(f"Found {len(markets)} markets. Fetching candle data...")

		for j, market in enumerate(markets):
			# 2. Robust Time Parsing
			# Try to get close time, fall back to expiration if needed
			time_val = market.get('close_time') or market.get('expiration_time')
			if not time_val:
				continue

			try:
				end_ts = int(dateutil.parser.parse(time_val).timestamp())
			except Exception as e:
				print(f"Date parse error: {e}")
				continue

			# 3. FIX: Set Window to 48 Hours (not 3 seconds)
			# 60 seconds * 60 minutes * 48 hours
			start_ts = end_ts - (60 * 60 * 48)

			try:
				resp = client.get_market_candlesticks(
					series_ticker=ticker,
					ticker=market['ticker'],
					start_ts=start_ts,
					end_ts=end_ts,
					period_interval=1 # 1 minute candles
				)

				candles = resp.get('candlesticks', [])

				if candles:
					# Flatten the data structure: 1 row per candle
					for candle in candles:

						# 4. FIX: Use correct keys for nested OHLC data
						# 'yes_bid' contains {open, high, low, close}, not 'price'
						bid_data = candle.get('yes_bid') or {}
						ask_data = candle.get('yes_ask') or {}
						price_data = candle.get('price') or {}

						row = {
							"series_ticker": ticker,
							"market_ticker": market['ticker'],
							"market_status": market.get('status'),
							"time_since_open_ts": -int(dateutil.parser.parse(market.get('open_time')).timestamp())+candle.get('end_period_ts'),

							# Trade Data (Executed)
							# Using np.nan ensures Pandas treats this as "Missing Data" rather than the object "None"
							"open": price_data.get('open', np.nan),
							"high": price_data.get('high', np.nan),
							"low": price_data.get('low', np.nan),
							"close": price_data.get('close', np.nan),
							"volume": candle.get('volume', 0), # Volume usually defaults to 0, not NaN

							# Order Book Data (Bid/Ask)
							# Fixed syntax error: changed bid_data('high') to bid_data.get('high', np.nan)
							"yes_bid_open": bid_data.get('open', np.nan),
							"yes_bid_low": bid_data.get('low', np.nan),
							"yes_bid_high": bid_data.get('high', np.nan),
							"yes_bid_close": bid_data.get('close', np.nan),

							"yes_ask_open": ask_data.get('open', np.nan),
							"yes_ask_low": ask_data.get('low', np.nan),
							"yes_ask_high": ask_data.get('high', np.nan),
							"yes_ask_close": ask_data.get('close', np.nan),

							# Spread Calculations
							# Since we default to np.nan, the math below is safe.
							# (e.g., 50 - np.nan = np.nan)
							"open_spread": ask_data.get("open", np.nan) - bid_data.get("open", np.nan),
							"close_spread": ask_data.get("close", np.nan) - bid_data.get("close", np.nan)
						}
						all_candle_data.append(row)

			except Exception as e:
				print(f"❌ Error fetching candles for {market['ticker']}: {e}")
				continue

	# Convert list of dicts to DataFrame
	df = pd.DataFrame(all_candle_data)

	if not filename.endswith('.csv'):
		filename += '.csv'

	print(f"Successfully processed {len(df)} candle records.")
	df.to_csv(filename, index=False)
	return df

# Usage:
# Increase limit to ensure you get markets
fetch_market_candles_to_csv(["KXNBAGAME"], "nba_candles", limit=1)

(1/1) Fetching market data for Series: KXNBAGAME...
Found 1 markets. Fetching candle data...
Successfully processed 2390 candle records.


,series_ticker,market_ticker,market_status,time_since_open_ts,open,high,low,close,volume,yes_bid_open,yes_bid_low,yes_bid_high,yes_bid_close,yes_ask_open,yes_ask_low,yes_ask_high,yes_ask_close,open_spread,close_spread
0,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32220,NaN,NaN,NaN,NaN,0,65,65,65,65,74,73,74,73,9,8
1,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32340,NaN,NaN,NaN,NaN,0,65,65,65,65,73,73,73,73,8,8
2,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32400,73.0,73.0,73.0,73.0,39,65,65,73,73,73,73,74,74,8,1
3,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32460,73.0,73.0,73.0,73.0,8,73,73,73,73,74,74,74,74,1,1
4,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32520,73.0,74.0,73.0,73.0,39,73,73,73,73,74,74,74,74,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,204060,1.0,1.0,1.0,1.0,3114,0,0,0,0,1,1,1,1,1,1
2386,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,204120,1.0,1.0,1.0,1.0,9459,0,0,0,0,1,1,1,1,1,1
2387,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,204180,1.0,1.0,1.0,1.0,166,0,0,0,0,1,1,1,1,1,1
2388,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,204240,NaN,NaN,NaN,NaN,0,0,0,0,0,1,1,1,1,1,1


In [27]:
df.head()

,series_ticker,market_ticker,market_status,time_since_open_ts,open,high,low,close,volume,yes_bid_open,yes_bid_low,yes_bid_high,yes_bid_close,yes_ask_open,yes_ask_low,yes_ask_high,yes_ask_close,open_spread,close_spread
0,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32220,NaN,NaN,NaN,NaN,0,65,65,65,65,74,73,74,73,9,8
1,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32340,NaN,NaN,NaN,NaN,0,65,65,65,65,73,73,73,73,8,8
2,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32400,73.0,73.0,73.0,73.0,39,65,65,73,73,73,73,74,74,8,1
3,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32460,73.0,73.0,73.0,73.0,8,73,73,73,73,74,74,74,74,1,1
4,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,32520,73.0,74.0,73.0,73.0,39,73,73,73,73,74,74,74,74,1,1


In [30]:
df.sort_values('time_since_open_ts', ascending=False).head(5)

,series_ticker,market_ticker,market_status,time_since_open_ts,open,high,low,close,volume,yes_bid_open,yes_bid_low,yes_bid_high,yes_bid_close,yes_ask_open,yes_ask_low,yes_ask_high,yes_ask_close,open_spread,close_spread
2389,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,204300,1.0,1.0,1.0,1.0,333,0,0,0,0,1,1,100,100,1,100
2388,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,204240,NaN,NaN,NaN,NaN,0,0,0,0,0,1,1,1,1,1,1
2387,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,204180,1.0,1.0,1.0,1.0,166,0,0,0,0,1,1,1,1,1,1
2386,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,204120,1.0,1.0,1.0,1.0,9459,0,0,0,0,1,1,1,1,1,1
2385,KXNBAGAME,KXNBAGAME-25DEC03MIADAL-MIA,finalized,204060,1.0,1.0,1.0,1.0,3114,0,0,0,0,1,1,1,1,1,1
